# Computational Drug Discovery 

In [ ]:
#Importing the necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

## Data Collection

### Searching for target protein (Dengue Virus)

In [ ]:
#General settings, increasing displayed rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
#Target search for dengue virus
target = new_client.target
target_query = target.search("dengue virus")
targets = pd.DataFrame.from_dict(target_query)
targets

### Selecting and retrieving bioactivity data of nonstructural protein 5 (seventh entry)

[Nonstructural protein 5](https://www.sciencedirect.com/topics/medicine-and-dentistry/nonstructural-protein-5) is a component of the dengue virus RNA genome, encoding for a methyltransferase (MTase) at the N-terminal, while the C-terminal encodes for the RNA-dependent RNA polymerase.

In [ ]:
#selecting the target protein to the variable "selected_target"
selected_target = targets.target_chembl_id[6]
selected_target

We will only focus on the bioactivity, reported as the half maximal inhibitory concentration (IC50). IC50 indicates how much of a particular inhibitory substance (e.g. drug) is needed to inhibit, in vitro, a given biological process or biological component by 50%.
IC50 is given as nanomolar (nM) units.

In [ ]:
#getting IC50 data of the target protein
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
#creating a dataframe from the filtered data + general information
df = pd.DataFrame.from_dict(res)
df.info()

In [ ]:
#Getting the first five rows to sample the data
df.head(5)

In [ ]:
#writing the dataframe to a csv file
df.to_csv("bioactivity_data_raw.csv", index=False)

### Correcting missing data

In [ ]:
df2 = df[df.standard_value.notna()]
df2

It looks like the dataset does not contain missing data for the standard values. 

###  Pre-processing of data

In this dataframe, bioactivity is represented as standard value of IC50. A compound with a value below 1,000 nM will be considered **active**, while those above 10,000 nM are considered **inactive**. Additionally, values between 1,000 and 10,000 nm are called **intermediate**.

In [ ]:
#Labelling compound classes of bioactivity
bioactivity_class= []
for i in df2.standard_value:
    if float(i) <= 1000:
        bioactivity_class.append("active")
    elif float(i) >= 10000:
        bioactivity_class.append("inactive")
    else:
        bioactivity_class.append("intermediate")

In [ ]:
#Selecting only the id, chemical structure and standard value of the compounds
selection = ["molecule_chembl_id", "canonical_smiles", "standard_value"]
df3 = df2[selection]
df3

Before concatenating the bioactivity classes, it needs to be converted to a pandas series. It is currently a list, and the function pd.concat can only work with series and pandas dataframes.

In [ ]:
#Converting the bioactivity class list to a pandas series
bioactivity_class = pd.Series(bioactivity_class, name="bioactivity_class")

In [ ]:
#Concatenating df3 with class definition
df4 = pd.concat([df3, bioactivity_class], axis=1)
df4

In [ ]:
#Identifying duplicate molecules
number = df4.molecule_chembl_id.unique()
number

There are currently 62 unique chembl ids, and 65 rows. This means that there must be a single duplicate chembl id, since there are two rows with NaN values. This is CHEMBL1418094. We can therefore drop a duplicate value of CHEMBL1418094.

In [ ]:
df5= df4.drop([62])
df5

We can additionally see that there are currently a few molecules with either NaN, or incorrect values for their bioactivity class. However, we can manually discern their bioactivity class from their standard values. We can proceed to add these in the dataframe. 

In [ ]:
df5.bioactivity_class[55]= "inactive"
df5.bioactivity_class[56]= "inactive"
df5.bioactivity_class[61]= "intermediate"
df5.bioactivity_class[63]= "active"
df5.bioactivity_class[64]= "active"
df5

In [ ]:
#Removing missing data
df6=df5[df4.standard_value.notna()]
df6

Finally, we can write the newly preprocessed data into a csv file.

In [ ]:
df6.to_csv("bioactivity_data_preprocessed.csv", index=False)

In [ ]:
! ls -l

We've now created the raw and preprocessed data of nonstructural protein 5.

## Exploratory Data Analysis

### Loading the preprocessed data

In [ ]:
new_df = pd.read_csv('bioactivity_data_preprocessed.csv')

### Installing conda and rdkit

[Conda](https://docs.conda.io/en/latest/) is a open source package management system, assisting in installing packages. On the other hand, [rdkit](https://github.com/rdkit/rdkit) is a collection of cheminformatics and machine-learning software written in C++ and Python; it allows you to compute molecular descriptors for the previously compiled data. 

We will use these tools in this section.

- conda install -c conda-forge rdkit

### Calculating Lipinski descriptors

A scientist named Christopher A. Lipinski, devised a rule of thumb to evaluate the **druglikeness** of a chemical compound. The result would then determine whether the compound would constitute as a suitable, orally active drug in humans. This is dubbed as [Lipinski's rule of five](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2728118/). The overall druglikeness is based on five molecular properties, namely the absorption, distribution, metabolism, and excretion ("ADME").

The Lipinski's Rule states the following:

- Molecular weight < 500 Dalton
- Octanol-water partition coefficient (LogP) < 5
- Hydrogen bond donors < 5
- Hydrogen bond acceptors < 10

In [ ]:
#Importing the required libraries
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [ ]:
#Compute descriptors 
def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [ ]:
df_lipinski = lipinski(df3.canonical_smiles)
df_lipinski

We can now proceed with combining the two dataframes "df_lipinski" and "new_df" to obtain a better overview of the molecular data.

In [ ]:
df_combined = pd.concat([new_df, df_lipinski], axis=1)

In [ ]:
df_combined

In [ ]:
#Dropping row 62 since that was a copy of CHEMBL418052
df_combined2= df_combined.drop([62])
df_combined2

### Converting IC50 to pIC50

[pIC50](https://www.collaborativedrug.com/what-is-pic50-2/) is considered to be the negative logarithmic of IC50 (-log10(IC50)). We perform this conversion in order to make the IC50 data more uniformly distributed. 

Here, we will utilize the custom function pIC50(). It  will accept a DataFrame as input, followed by:
- Taking the IC50 values from the standard_value column and converting it from nM to M by multiplying the value by 10^-9
- Taking the molar value and applying -log10
- Deleting the standard_value column and creating a new pIC50 column

In [ ]:
def pIC50(input):
    pIC50 = []

    for i in input['standard_value']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value', 1)
        
    return x

One thing to note however; a value greater than 100,000,000 will cause its negative logarithmic value to turn negative. Normally, you would need to cap the values to a limit of 100,000,000 in order to prevent this occurrence. 

In [ ]:
df_combined2.standard_value.describe()

This dataset seems to lack any values above 100,000,000. We can therefore omit the capping process, and proceed with the conversion of IC50 to pIC50.

In [ ]:
df_final = pIC50(df_combined2)
df_final

In [ ]:
df_final.describe()

In [ ]:
df_final.info()

To allow a more simple comparison between active and inactive compounds, we are going to remove the "intermediate" bioactivity class. 

In [ ]:
df_2class = df_final[df_final.bioactivity_class != 'intermediate']
df_2class